# Implied Volatility

##### Newtons method

In [ ]:
def implied_volatility(price: float, S: int, K: int, T: float, r: float, otype: str, q = .02) -> float:

    if T >= 1:
        T /= 365

    elif T == 0:
        T = .5 / 365
 
    xnew: float = .20
    xold: float = 1 - xnew
    for i in range(100):
        if abs(xnew - xold) < .001:
            
            return round(xnew, 3) * 100
        else:
            xold = xnew

            if otype == "call":
                xnew = xnew - ((call(S, K, T, xold, r, q) - price) / vega(S, K, T, xold, r, q))

            elif otype == 'put':
                xnew = xnew - ((put(S, K, T, xold, r, q) - price) / vega(S, K, T, xold, r, q))
            #print(f"xnew is {xnew}")
    return False

##### Implied Volatility Using Scipy Root
##### - We can experiment and use other option pricing models with the root instead of bsm

In [2]:
from scipy.optimize import root
from scipy.stats import norm
import numpy as np
import pandas as pd

def bs_call(S_0,K,T,sigma,r):
    d_1 = (np.log(S_0/K) + T*(r + (sigma**2)/2))/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)
    return S_0*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)

def get_implied_vol_bs(S_0,K,T,r,observed_px, initial_guess):
    root_fn = lambda x: bs_call(S_0,K,T,x,r) - observed_px
    return root(root_fn,initial_guess)['x'][0]

In [10]:
get_implied_vol_bs(100, 105, 1/12, .01, 1.5, .2)

0.28894745076596384

##### Implied Volatility From py_vollib Library

In [ ]:
from py_vollib.black_scholes.implied_volatility import implied_volatility

In [ ]:
# Example

implied_volatility(cev_call(S_0,K,T,vol,b,r),S_0,K,T,r,'c')

### Implied Volatility from yves hilspsich derivatives analytics

In [ ]:
def bsm_call_imp_vol(S0, K, T, r, C0, sigma_est, it=100):

    for i in range(it):
        sigma_est -= ((bsm_call_value(S0, K, T, r, sigma_est) - C0)
                        / bsm_vega(S0, K, T, r, sigma_est))
    return sigma_est

### This is also from the book derivatives analytics except this uses scipy fsolve to find IV


In [ ]:
def imp_vol(C0, sigma_est=0.2):
    ''' Return implied volatility given option price. '''
    option = call_option(S0, K, t, M, r, sigma_est)
    option.update_ttm()
    def difference(sigma):
        option.sigma = sigma
        return option.value() - C0
    iv = fsolve(difference, sigma_est)[0]
    return iv

##### SABR Normal Vol

In [ ]:
def sabr_normal_vol(S_0,K,T,sigma_0,alpha,beta,rho):
    c = lambda x: x**beta
    c_prime = lambda x: beta*(x**(beta-1))
    c_prime_prime = lambda x: beta*(beta-1)*(x**(beta-2))
    S_mid = (S_0 + K)/2
    gamma_1 = c_prime(S_mid)/c(S_mid)
    gamma_2 = c_prime_prime(S_mid)/c(S_mid)
    zeta = alpha*(S_0**(1-beta) - K**(1-beta))/(sigma_0 * (1-beta))
    epsilon = T*(alpha**2)
    delta = np.log( (np.sqrt(1 - 2*rho*zeta + zeta**2) + zeta - rho)/(1-rho) )

    factor = alpha*(S_0 - K)/(delta)
    term_1 = ((2*gamma_2 - gamma_1**2)/24)* (sigma_0*c(S_mid)  / alpha)**2
    term_2 = rho*gamma_1*sigma_0*c(S_mid)/(4*alpha)
    term_3 = (2-3*(rho**2))/24
    return factor*(1 + epsilon*(term_1 + term_2 + term_3))

# Option Price Formula

##### Bachelier Formula Call

In [1]:
import numpy as np
from scipy.stats import norm

def bachelier_call(S_0,K,T,sigma,r):
    d_plus = (S_0*np.exp(r*T) - K)/(sigma*np.sqrt(T))
    return np.exp(-r*T)*sigma*np.sqrt(T)*(d_plus*norm.cdf(d_plus) + norm.pdf(d_plus))

print(bachelier_call(S_0 = 100, K = 105, T = 0.5, sigma = 30, r = 0.02))

6.549163351317259


##### Black Scholes Merons Put & Call

In [2]:
def put(self, S: float, K: float, T: float, sigma: float, r: float = 0.06, q: float = 0) -> float:
    d1: float = (np.log(S/K) + (r - q + ((sigma**2)/2)) * T) / (sigma*np.sqrt(T))
    d2: float = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * norm.cdf(-d2) - S*np.exp(-q*T) * norm.cdf(-d1)

def call(self, S, K, T, sigma, r, q) -> float:
    d1 = (np.log(S/K) - (r - q + ((sigma**2)/2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

##### Monte Carlo Put & Call Price & Simulation

In [ ]:
def monte_carlo_simulation(S0, M, I, T, sigma, r) -> list:
    dt = T / M
    S = np.zeros((M + 1, I))
    S[0] = S0
    rn = np.random.standard_normal(S.shape)
    for t in range(1, M + 1):   # 1
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t])  # 2
    return S

def mc_call_price(S0, K, M, I, T, sigma, r) -> float:
    
    S: list = monte_carlo_simulation(S0, M, I, T, sigma, r)
    return np.exp(-r * T) * np.maximum(S[-1] - K, 0).mean()

def mc_put_price(S0, K, M, I, T, sigma, r) -> float:
    
    S: list = monte_carlo_simulation(S0, M, I, T, sigma, r)
    return np.exp(-r * T) * np.maximum(K - S[-1], 0).mean()

##### BSM For American Options Using a PDE

In [16]:
def solve_bs_pde(s0,smax,k,T,N,M,sig,r,p):
    
    ht = T/N
    hs = smax/M
    t = np.arange(0, T+ht, ht)
    s = np.arange(0, smax+hs, hs)
    
    d = 1-(sig**2)*(s**2)*ht/(hs**2)-r*ht
    l = 0.5*(sig**2)*(s**2)*ht/(hs**2)-r*s*ht/(2*hs)
    u = 0.5*(sig**2)*(s**2)*ht/(hs**2)+r*s*ht/(2*hs)
    
    A = np.matrix(np.zeros((M-1,M-1)))
    diag = d[1:]
    upperDiag = u[1:M-1]
    lowerDiag = l[2:M]
    for i in range(len(upperDiag)):
        A[i,i+1] = upperDiag[i]
        A[i+1,i] = lowerDiag[i]
    for i in range(M-1):
        A[i,i] = diag[i]
    vec_eigenvalue = np.linalg.eigvals(A)
    
    b = u[M-1]*(s[M]-k*np.exp(-r*(T-t)))
    #ba = u[M-1]*(s[M]-k)

    diff = s-k
    diff[diff<0]=0
    ter_c = np.matrix(diff[1:M]).T
    cont_val = ter_c

    for i in range(N, 1, -1):
        bb = np.append(np.zeros(M-2),b[i]).reshape(M-1,1)
        exercise_val =  np.maximum(s[1:-1]-k, np.zeros(M-1)).reshape(M-1,1)
        cont_val = A @ cont_val + bb
        # exercise if exercise value exceeds continuation value
        vec_c = np.maximum(cont_val, exercise_val)
        
    return vec_c

##### CEV Model

In [3]:
from scipy.stats import ncx2
import numpy as np

# ncx2 should be a chi squared distribution

def cev_call(S_0,K,T,sigma,beta,r):
    v = 1/(2*(1-beta))
    x_1 = 4*(v**2)*(K**(1/v))/((sigma**2) * T)
    x_2 = 4*(v**2)*((S_0*np.exp(r*T))**(1/v))/((sigma**2) * T)
    kappa_1 = 2*v + 2
    kappa_2 = 2*v
    lambda_1 = x_2
    lambda_2 = x_1
    return np.exp(-r*T)*((S_0*np.exp(r*T)*(1-ncx2.cdf(x_1,kappa_1,lambda_1))) - K*ncx2.cdf(x_2,kappa_2,lambda_2))

print(cev_call(S_0 = 100, K = 100, T = 0.5, sigma =4, r = 0.02, beta  = 0.5))

11.676110446149188


##### SABR Call Based On SABR Normal Vol & Bachelier Call

In [ ]:
def sabr_call(S_0,K,T,sigma_0,r,alpha,beta,rho):
    assert(S_0 != K)
    vol = sabr_normal_vol(S_0,K,T,sigma_0,alpha,beta,rho)
    return bachelier_call(S_0,K,T,vol,r)

##### Binomial Tree Call Option Price (Chris K Quant Book)

In [1]:
def binomial_tree_call_option_price(N, r, sigma, S_0, T, K) -> float:
    # T = time to maturity in years = 1 ex so 1 month is T = 1 / 12
    deltaT: float = T/N
    u: float = np.exp(sigma * np.sqrt(deltaT))
    d: float = 1/u
    p: float = (np.exp(r*deltaT) - d)/(u-d)

    priceTree: list[list[float]] = [n*[None] for n in range(1,N+2)]

    for i in range(N+1): #each time period
        for j in range(i+1): #each possible price within each time period
            priceTree[i][j] = S_0 *  (u**(j)) * (d**(i-j))

    callPriceTree: list[list[float]] = [n*[None] for n in range(1,N+2)]
    #terminal payoffs
    callPriceTree[-1] = [max(px-K,0) for px in priceTree[-1]]

    for i in range(N-1,-1,-1): #each time period starting from the end
        for j in range(i+1): #each price within each time period
            callPriceTree[i][j] = np.exp(-r*deltaT)*((1-p)*callPriceTree[i+1][j] + p*callPriceTree[i+1][j+1])

    return callPriceTree[0][0]

def binomial_tree_put_price(N, r, sigma, S0, T, K) -> float:
    delta_t: float = T/N
    u: float = np.exp(sigma * np.sqrt(delta_t))
    d: float = 1/u
    p: float = (np.exp(r*delta_t) - d)/(u-d)

    price_tree = [n*[None] for n in range(1, N + 2)]

    for i in range(N + 1):
        for j in range(i+1):
            price_tree[i][j] = S0 * (u**j) * (d**(i - j))

    put_price_tree: list[list[float]] = [n*[None] for n in range(1,N+2)]
    put_price_tree[-1] = [max(K - px,0) for px in price_tree[-1]]

    for i in range(N-1, -1, -1):
        for j in range(i+1):
            put_price_tree[i][j] = np.exp(-r*delta_t)*((1-p)*put_price_tree[i+1][j] + p*put_price_tree[i+1][j+1])

    return put_price_tree[0][0]

binomial_tree_put_price(100, .04, .3, 40, .5, 40)

NameError: name 'np' is not defined

In [6]:
binomial_tree_call_option_price(30, .04, .2, 500, 1/12, 501)

11.81351070694491

##### Call Price Heston FFT & Characteristic Equation

In [7]:
def heston_characteristic_eqn(u, sigma, k, p, s_0, r, t, theta, v_0):
    lambd = np.sqrt((sigma**2)*((u**2)+1j*u) + (k - 1j*p*sigma*u)**2) 
    omega_numerator = np.exp(1j*u*np.log(s_0)+1j*u*(r)*t+(1/(sigma**2))*k*theta*t*(k - 1j*p*sigma*u))
    omega_denominator = (np.cosh(0.5*lambd*t) + (1/lambd)*(k - 1j*p*sigma*u)*np.sinh(0.5*lambd*t))**((2*k*theta)/(sigma**2))
    phi = (omega_numerator/omega_denominator) * np.exp(-((u**2 + 1j*u)*v_0)/(lambd*(1/np.tanh(0.5*lambd*t)) + (k - 1j*p*sigma*u)))
    return phi

def calc_fft_heston_call_prices(alpha, N, delta_v, sigma, k, p, s_0, r, t, theta, v_0, K = None):
    #delta is the indicator function
    delta = np.zeros(N)
    delta[0] = 1 
    delta_k = (2*np.pi)/(N*delta_v)
    if K == None:
        #middle strike is at the money
        beta = np.log(s_0) - delta_k*N*0.5 
    else:
        #middle strike is K
        beta = np.log(K) - delta_k*N*0.5
    k_list = np.array([(beta +(i-1)*delta_k) for i in range(1,N+1) ])
    v_list = np.arange(N) * delta_v
    #building fft input vector
    x_numerator = np.array( [((2-delta[i])*delta_v)*np.exp(-r*t)  for i in range(N)] )
    x_denominator = np.array( [2 * (alpha + 1j*i) * (alpha + 1j*i + 1) for i in v_list] )
    x_exp = np.array( [np.exp(-1j*(beta)*i) for i in v_list] )
    x_list = (x_numerator/x_denominator)*x_exp* np.array([heston_characteristic_eqn(i - 1j*(alpha+1),sigma, k,p,s_0,r,t,theta, v_0) for i in v_list])
    #fft output
    y_list = np.fft.fft(x_list)
    #recovering prices
    prices = np.array( [(1/np.pi) * np.exp(-alpha*(beta +(i-1)*delta_k)) * np.real(y_list[i-1]) for i in range(1,N+1)] )
    return prices, np.exp(k_list)

##### Heston Europen option price Monte Carlo Simulation

In [14]:
def euro_payoffs(k, t, is_call, paths):
    st = paths[:, -1]
    return np.fmax(st - k, 0) if is_call else np.fmax(k - st, 0)

def c(s0=s0, nu0=nu0, theta=theta, t=t, r=r, kappa=kappa, xi=xi, rho=rho, n=n, N=N, k=k):
    np.random.seed(0)  # to get the same random numbers every time
    pathsS0, pathsNu = heston_simulate(s0, t, r, nu0, kappa, theta, xi, rho, n, N)
    euro_prc = np.exp(-r * t) * np.mean(euro_payoffs(k, t, True, pathsS0))
    return euro_prc

NameError: name 's0' is not defined

##### Pricing A Digital Call With BSM & Bachelier Using A Quadrature 

In [ ]:
def price_digital_call_quad(S_0,K,r,T,density_func, b, N):
    """ Using left riemann sum"""
    width = (b-K)/N
    nodes = np.arange(K,b,width)
    areas = [width * density_func(node) for node in nodes]
    px = np.exp(-r*T)*sum(areas)
    return px

#Black-Scholes
density_bs = lognorm(s = sigma_bs*np.sqrt(T), scale = np.exp(np.log(S_0)+ (r - 0.5*(sigma_bs**2))*T)).pdf
print("Black-Scholes price:",price_digital_call_quad(S_0 = S_0,K = K,r = r,T = T,density_func = density_bs,b = b,N = N))
#Black-Scholes price: 0.4641456578462726

#Bachelier
density_bach = norm(loc = S_0*np.exp(r*T),scale = sigma_bach*np.sqrt(T)).pdf
print("Bachelier price:",price_digital_call_quad(S_0 = S_0,K = K,r = r,T = T,density_func = density_bach,b = b,N = N))
#Bachelier price: 0.5039894228040073

# Simulations

##### Basic Monte Carlo Simulation (I thin geo brownian motion)
##### Note: I think I adjusted this so it matches python for finance version where the calc on the numpy array is vectorized

In [24]:
def monte_carlo_simulation_bs(S0, M, I, T, sigma, r) -> list:
    dt = T / M
    S = np.zeros((M + 1, I))
    S[0] = S0
    rn = np.random.standard_normal(S.shape)
    for t in range(1, M + 1):   # 1
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t])  # 2
    return S

##### Heston Simulation

In [ ]:
def heston_simu(S0, K, r, T, v0, sig, k, theta, rho, n, N):
    
    dt = T/N
    mean = (0,0)
    cov = [[dt,rho*dt],[rho*dt,dt]]
    st_vec = np.empty(n)
    vt_vec = np.empty(n)
    payoff_vec = np.empty(n)
    price_vec = np.empty(n)

    for i in range(n):
    
        x = np.random.multivariate_normal(mean,cov,(N,1))
        st = S0
        vt = v0
    
        for j in range(N):
            dst = r*st*dt + np.sqrt(vt)*st*x[j][0][0]
            dvt = k*(theta-vt)*dt + sig*np.sqrt(vt)*x[j][0][1]
            st += dst
            vt += dvt
            if vt<0:
                vt = -vt
    
        payoff_vec[i] = max(st-K,0)
        price_vec[i] = payoff_vec[i]*np.exp(-r*T)
        st_vec[i] = st
        vt_vec[i] = vt
        
    plt.hist(st_vec,bins=100)
    plt.title("Terminal stock price distribution")
    plt.xlabel("stock price")
    plt.ylabel("frequncy per " + str(n) + " trials")
    plt.show()

    return price_vec

##### Heston simulation (Python for Finance)

In [27]:
def heston_process(S0, r, v0, kappa, theta, sigma, rho, T, M, I):

    corr_mat = np.zeros((2, 2))
    corr_mat[0, :] = [1.0, rho]
    corr_mat[1, :] = [rho, 1.0]
    cho_mat = np.linalg.cholesky(corr_mat)

    dt = T / M

    ran_num = np.random.standard_normal((2, M + 1, I))

    S = np.zeros_like(ran_num[0])
    v = np.zeros_like(ran_num[0])

    S[0] = S0
    v[0] = v0

    for t in range(1, M + 1):
        ran = np.dot(cho_mat, ran_num[:, t, :])

        v[t] = (v[t - 1] +
                kappa * (theta - np.maximum(v[t - 1], 0)) * dt +
                sigma * np.sqrt(np.maximum(v[t - 1], 0)) *
                np.sqrt(dt) * ran[1])
        
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt +
                                np.sqrt(v[t]) * ran[0] * np.sqrt(dt))
        
    return S

def mc_call_price_heston(paths, K: int, T: float, r: float) -> float:
    
    return np.exp(-r * T) * np.maximum(paths[-1] - K, 0).mean()

# note we are going to turn this into a more general structure where we pass in a process and price it

paths = heston_process(100, .05, .1, 3, .25, .1, .6, .5, 100, 1000)
mc_call_price_heston(paths, 100, .5, .02)

14.192173393958564

##### Variance Gamma Simulation

In [13]:
def var_gamma(sigma, v, theta, S0, T, r, q, n, N):
    
    dt = T/N
    w = np.log(1-theta*v-0.5*v*sigma**2)/v
    price_vec = np.empty(n)
    
    for i in range(n):
        
        lns0 = np.log(S0)
        xt = 0
        
        for j in range(N):
            
            gamma = np.random.gamma(dt/v, v)
            z = np.random.normal(0,1)
            xt += theta*gamma + sigma*np.sqrt(gamma)*z

            Tj = dt*(j+1)
            lnst = lns0 + (r-q+w)*Tj + xt

        price = np.exp(lnst)
        price_vec[i] = price
        
    return price_vec

# This is the variance gamma I made based on vectorization

def variance_gamma_simulation(sigma, v, theta, S0, T, r, q, I, M):

    dt = T / M
    w = np.log(1-theta*v-0.5*v*sigma**2)/v
    S = np.zeros((M + 1, I))

    lns0 = np.log(S0)
    S[0] = np.log(S0)
    xt = 0

    rn = np.random.standard_normal(S.shape)
    gamma = np.random.gamma(dt/v, v, S.shape)

    j = np.array([i for i in range(I)])

    Tj = dt*(j+1)
    

    for t in range(1, M + 1):

        xt += theta*gamma[t] + sigma*np.sqrt(gamma[t])*rn[t]

        #Tj = dt*(j+1)

        S[t] = S[0] + (r-q+w)*Tj + xt

    price = np.exp(S)
    return price

##### SABR Euler Discretization

In [ ]:
def sabr_simulation(self, fixed_seed=True, day_count=365.):
    ''' Generates Monte Carlo Paths using Euler discretization.
    '''
    if self.time_grid is None:
        self.generate_time_grid()
    M = len(self.time_grid)
    I = self.paths
    paths = np.zeros((M, I))
    va = np.zeros_like(paths)
    va_ = np.zeros_like(paths)
    paths[0] = self.initial_value
    va[0] = self.alpha
    va_[0] = self.alpha
    if self.correlated is False:
        sn1 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
    else:
        sn1 = self.random_numbers

    # pseudo-random numbers for the stochastic volatility
    sn2 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)  # this is just np.random.standard_normal()

    for t in range(1, len(self.time_grid)):
        dt = (self.time_grid[t] - self.time_grid[t - 1]).days / day_count
        square_root_dt = np.sqrt(dt)
        if self.correlated is False:
            ran = sn1[t]
        else:
            ran = np.dot(self.cholesky_matrix, sn1[:, t, :])
            ran = ran[self.rn_set]
        rat = np.array([ran, sn2[t]])
        rat = np.dot(self.leverage, rat)

        va_[t] = va_[t - 1] * (1 + self.vol_vol * square_root_dt * rat[1])
        va[t] = np.maximum(0, va_[t])

        F_b = np.abs(paths[t - 1]) ** self.beta
        p = paths[t - 1] + va_[t] * F_b * square_root_dt * rat[0]
        if (self.beta > 0 and self.beta < 1):
            paths[t] = np.maximum(0, p)
        else:
            paths[t] = p

    self.instrument_values = paths
    self.volatility_values = np.sqrt(va)

##### Square root diffusion (I think cox ingersol ross) simulation
##### ie CIR process I think

In [3]:
x0 = 0.05
kappa = 3
theta = .02
sigma = .1
I = 10000
M = 50
T = .5
dt = T / M

In [5]:
import numpy as np

def srd_euler():
    xh = np.zeros((M + 1, I))
    x = np.zeros_like(xh)
    xh[0] = x0
    x[0] = x0
    for t in range(1, M + 1):
        xh[t] = (xh[t - 1] + kappa * (theta - np.maximum(xh[t - 1], 0)) * dt +
                             sigma * np.sqrt(np.maximum(xh[t - 1], 0)) *
                             np.sqrt(dt) * np.random.standard_normal(I))
    
    x = np.maximum(xh, 0)
    return x

# Characteristic Equations

##### Heston

In [8]:
def heston_characteristic_eqn(u, sigma, k,p,s_0,r,t,theta, v_0):
    lambd = np.sqrt((sigma**2)*((u**2)+1j*u) + (k - 1j*p*sigma*u)**2) 
    omega_numerator = np.exp(1j*u*np.log(s_0)+1j*u*(r)*t+(1/(sigma**2))*k*theta*t*(k - 1j*p*sigma*u))
    omega_denominator = (np.cosh(0.5*lambd*t) + (1/lambd)*(k - 1j*p*sigma*u)*np.sinh(0.5*lambd*t))**((2*k*theta)/(sigma**2))
    phi = (omega_numerator/omega_denominator) * np.exp(-((u**2 + 1j*u)*v_0)/(lambd*(1/np.tanh(0.5*lambd*t)) + (k - 1j*p*sigma*u)))
    return phi

# Parameter Calibrations

##### CEV Model Parameter Calibration

In [11]:
from scipy.optimize import minimize

def sum_squares_cev(beta,S_0,sigma,r,call_prices, strikes, expiries):
    sum = 0
    for price, strike, expiry in zip(call_prices, strikes, expiries):
      sum += (price - cev_call(S_0,strike,expiry,sigma,r,beta))**2
    return sum

def find_beta_sigma_cev(S_0,r,call_prices, strikes, expiries, guess = [0.9,0.4],bounds = ((0.001,None),(0.001,None)),tol=1e-10):
    """
    call_prices, strikes, and expiries are arrays of equal length with each index corresponding to one option
    For example, if the first elements of each array are 10, 100, and 1 respectively, this corresponds to an option
    with price 10, strike 100, and 1 year to expiry
    """
    #first element is beta, second element is sigma
    unique_expiries = np.unique(expiries)
    calibrated_beta_sigma = {} #this is where the results will go for each expiry
    for expiry in unique_expiries:
      #how many times this expiry appears
      expiries_list = expiry*np.ones(np.sum(expiries == expiry)) 
      #where it appears
      indices = np.where(expiries == expiry)[0] 
      #what are the strikes for this expiry
      strikes_for_expiry = np.array(strikes)[indices] 
      #what are the market prices for this expiry
      prices_for_expiry = np.array(call_prices)[indices] 
      #the function to be minimized
      opt_func = lambda x: sum_squares_cev(x[0],S_0,x[1],r,prices_for_expiry, strikes_for_expiry, expiries_list) 
      #calibrated values
      beta, sigma = minimize(opt_func,guess, bounds = bounds ,method = 'SLSQP', tol=tol)['x'] 
      #saving the results
      calibrated_beta_sigma[expiry] = [beta, sigma]
    return calibrated_beta_sigma

##### Calibrated SABR

In [17]:
def asymptotic_normal_vol(T,K,F0,sigma0,alpha,beta,rho):
    
    Fmid = (F0 + K)/2
    zeta = alpha/(sigma0 * (1-beta)) * (F0 ** (1-beta) - K **(1-beta))
    eps = T*alpha**2
    delta = np.log((np.sqrt(1-2 * rho * zeta + zeta**2) + zeta - rho)/(1-rho))
    gamma1 = beta/Fmid
    gamma2 = beta * (beta -1)/Fmid**2
    
    parta = alpha * (F0-K)/delta
    partb1 = (2*gamma2 - gamma1**2)/24 * (sigma0 * Fmid ** beta /alpha)**2
    partb2 = rho * gamma1/4 * sigma0 * Fmid **beta/alpha
    partb3 = (2-3 *rho **2)/24
    partb = (1+(partb1 + partb2 + partb3) * eps)
    
    return parta*partb

def rootfind_helper(sabr_params,T,K_list,F0,sigma_list):
    
    sigma0,alpha,rho = sabr_params
    beta = 0.5
    MSE = 0
    
    for i in range(len(K_list)):
        
        diff = asymptotic_normal_vol(T,K_list[i],F0,sigma0,alpha,beta,rho) - sigma_list[i]
        
        MSE += diff**2
        
    return MSE

def calibrate_sabr(T_list,K_table,F0_list,sigma_table):
    
    init_guess = [0.1,0.1,-0.1]
    
    params = np.zeros((len(T_list,3)))
    
    for T,i in enumerate(T_list):
        
        
        opt = scipy.minimize(rootfind_helper,init_guess,args = (T,K_table[i], F0_list[i],sigma_table[i]), method = 'SLSQP', bounds = ((0.01,1.5),(0,1.5),(-1,1)))
        params[i] = opt.x
    
    return params

# Implied Distribution

##### We use bsm for our call price in these, note we can use any model tho

In [3]:
from scipy.stats import norm
import numpy as np

def callPx(S, K, r, sigma, T, q = 0) -> float:
    d1 = (np.log(S/K) - (r - q + ((sigma**2)/2)) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

##### Breeden Litzenberger for risk neutral density from butterfly

In [4]:
import math
from scipy.stats import norm
from scipy import interpolate

def breeden_litzenberger(strikes, vols, s_0, rf, tau, dk):
    #use linear interpolation of implied volatility
    vol_interp = interpolate.interp1d(strikes, vols, fill_value="extrapolate")

    #loop through arrays and use Breeden-Litzenberger to find Risk Neutral Density at each point
    phis = np.full(len(strikes), 0.00)
    for zz, strike_zz in enumerate(strikes):
        px_up = callPx(s_0, strike_zz + dk, rf, vol_interp(strike_zz + dk), tau)
        px = callPx(s_0, strike_zz, rf, vol_interp(strike_zz), tau)
        px_dn = callPx(s_0, strike_zz - dk, rf, vol_interp(strike_zz - dk), tau)

        numer = (px_up - 2 * px + px_dn)
        denom = (dk * dk)
        phis[zz] = numer / denom

    return phis


s0 = 100.0
r = 0.0
exp_t = 0.25
upper_k = 150.0
lower_k = 50.0
n_points = 10001

dk = (upper_k - lower_k) / n_points

ks = np.linspace(lower_k, upper_k, n_points)
ivs = np.full(n_points, 0.2)

rnd = breeden_litzenberger(ks, ivs, s0, r, exp_t, dk)
print(rnd)

[-2.13205455e-10 -7.10684851e-11  2.13205455e-10 ...  5.51289640e-06
  5.49735954e-06  5.48186483e-06]


### Weighted Monte Carlo for risk neutral density extraction

In [5]:
import numpy as np
import cvxpy as cp
import math

def weighted_mc(call_strikes, impl_vols, s_0, tau, rf, num_nodes, lower_k, upper_k):
    nodes = np.linspace(lower_k, upper_k, num_nodes)

    x = cp.Variable(shape=num_nodes)
    obj = cp.Maximize(cp.sum(cp.entr(x)))
    
    constraints = [0.0 <= x, cp.sum(x) == 1, cp.sum(x * nodes) == s0 * math.exp(rf * tau)]

    threshold = s_0 * 0.005
    for zz, call_strike_zz in enumerate(call_strikes):
        payoffs = np.exp(-rf*tau) * np.maximum(nodes - call_strike_zz, 0.0)
        call_prices_zz = callPx(s_0, call_strike_zz, rf, impl_vols[zz], tau)
        constraints.append(cp.sum(x * payoffs) <= call_prices_zz + threshold)
        constraints.append(cp.sum(x * payoffs) >= call_prices_zz - threshold)

    prob = cp.Problem(obj, constraints)
    prob.solve(verbose=True)

    return x.value

s0 = 100.0
r = 0.0
exp_t = 0.25
upper_k = 150.0
lower_k = 50.0
n_points = 50

ks = np.linspace(lower_k, upper_k, n_points)
ivs = np.full(n_points, 0.2)

wmc_probs = weighted_mc(ks, ivs, s0, exp_t, r, n_points, lower_k, upper_k)
print(wmc_probs)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 16 04:38:45 PM: Your problem has 50 variables, 152 constraints, and 0 parameters.
(CVXPY) May 16 04:38:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 16 04:38:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 16 04:38:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 16 04:38:45 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 16 04:38:45 PM: Compiling problem (target solver=CLARABEL)

c:\Users\dibia\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
c:\Users\dibia\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so 